# Computing and creating the flights_aircrafts table and the optimal flights table

In [1]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime
# Package to get flight aircraft info
from traffic.core import Flight

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

def sigmoid(x, mu = 0, sig = 1):
    return 1/(1+np.exp(-(x - mu)/sig))

# Time Integral
def integrate_time(state_vectors):
    return state_vectors['time'].iloc[-1] - state_vectors['time'].iloc[0]

def integrate_fuel(state_vectors):
    return state_vectors['used_fuel'].iloc[-1]

# Wind Integral
def integrate_wind(state_vectors):
    speed_of_sound = 666.739
    return np.sum(state_vectors['sknt'])/(speed_of_sound*len(state_vectors))

# Air Density Integral
def integrate_air_density(state_vectors):
    sea_level_density = 1.204
    return np.sum(state_vectors['air_density'])/(sea_level_density*len(state_vectors))

def integrate_air_pressure(state_vectors):
    sea_level_pressure = 1013.25
    return np.sum(state_vectors['air_pressure'])/(sea_level_pressure*len(state_vectors))

def integrate_clouds(state_vectors):
    max_clouds = 1
    return np.sum(state_vectors['clouds'])/(max_clouds*len(state_vectors))

def integrate_severity(state_vectors):
    max_severity = 1
    return np.sum(state_vectors['severity'])/(max_severity*len(state_vectors))


def integrate_distance(state_vectors):
    d = 0
    for row_a, row_b in zip(state_vectors[:-1].itertuples(), state_vectors[1:].itertuples()):
        d += utils.haversine_distance(row_a.lat, row_a.lon, row_b.lat, row_b.lon)
    return d

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

In [31]:
flights_database = '../data/flight/KDEN_KSEA_2023-01-01_2023-01-31.sqlite'

conn = sqlite3.connect(flights_database)

cursor = conn.cursor()

flights = pd.read_sql_query(f"""
    SELECT *
    FROM flights
    ;""",conn)

aicraft_db = {'icao24':[], 'registration':[], 'typecode':[]}
for icao24 in flights['icao24']:
    clear_output(wait=True)
    print(icao24)
    df = pd.DataFrame({'icao24':[icao24], 'timestamp':[0]})
    ff = Flight(df)
    try:
        aicraft_db['icao24'] += [ff.aircraft['icao24']]
        aicraft_db['registration'] += [ff.aircraft['registration']]
        aicraft_db['typecode'] += [ff.aircraft['typecode']]
    except:
        aicraft_db['icao24'] += ['Other']
        aicraft_db['registration'] += ['Other']
        aicraft_db['typecode'] += ['Other']

cursor.execute("DROP TABLE IF EXISTS flights_aircraft;")

# Create the new table
create_table_query = f'''
    CREATE TABLE flights_aircraft (
        icao24 TEXT PRIMARY KEY,
        registration TEXT,
        typecode TEXT
    );
'''
# Create the new table if it doesn't exist
cursor.execute(create_table_query)

aicraft_db = pd.DataFrame(aicraft_db)

aicraft_db['typecode'] = aicraft_db['typecode'].apply(lambda x: 'Other' if x == '' else x)

for index, row in aicraft_db.iterrows():
    # Preparing the data to be inserted
    new_columns = ['icao24', 'registration', 'typecode']
    insert_data = tuple(row[col] for col in new_columns)

    # Creating query to insert new values
    insert_query = f'''
        INSERT INTO flights_aircraft ({', '.join(new_columns)})
        VALUES ({', '.join('?' * len(insert_data))})
        ON CONFLICT(icao24) DO UPDATE SET
        {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
    '''
    print(insert_data)
    cursor.execute(insert_query, insert_data)
        
    conn.commit()
conn.close()
print("Done")

a5dee3
('ada167', 'N978AK', 'B39M')
('a923a8', 'N68822', 'B739')
('aacd50', 'N79521', 'B738')
('abb19c', 'N8523W', 'B738')
('a3e55a', 'N350FR', 'A20N')
('a326d1', 'N302AS', 'B739')
('a01037', 'N103DU', 'BCS1')
('ab1644', 'N813UA', 'A319')
('a2b8c5', 'N27477', 'B739')
('ad01ab', 'N937WN', 'B737')
('a34e9d', 'N312DU', 'BCS3')
('a6ca7f', 'N537AS', 'B738')
('a42323', 'N366FR', 'A20N')
('ad0f29', 'N940WN', 'B737')
('a07b8f', 'N130DU', 'BCS1')
('a9238a', 'N68817', 'B739')
('ad01ab', 'N937WN', 'B737')
('a34e9d', 'N312DU', 'BCS3')
('a6ca7f', 'N537AS', 'B738')
('a42323', 'N366FR', 'A20N')
('ad0f29', 'N940WN', 'B737')
('a07b8f', 'N130DU', 'BCS1')
('a9238a', 'N68817', 'B739')
('a1c868', 'N214AK', 'B739')
('a54aae', 'N440AS', 'B739')
('a7016c', 'N550WN', 'B737')
('a32fd5', 'N304SY', 'E75L')
('a52898', 'N431UA', 'A320')
('abf2ea', 'N8692F', 'B738')
('a5fd2d', 'N485WN', 'B737')
('a35254', 'N313DU', 'Other')
('a3117c', 'N297WN', 'B737')
('a73220', 'N563AS', 'B738')
('a7a90e', 'N593FE', 'MD11')
('a07b

## Assigning optimal flight for each aircraft type

In [49]:
conn = sqlite3.connect(flights_database)

cursor = conn.cursor()

query = f"""
    SELECT fs.flight_id, fi.fuel, fa.typecode
    FROM flights AS fs
    JOIN flights_integrals AS fi ON fs.flight_id = fi.flight_id
    LEFT JOIN flights_aircraft AS fa ON fs.icao24 = fa.icao24
;"""

flights = pd.read_sql_query(query, conn)

unique_aircrafts = np.unique(flights['typecode'])

optimal_flights = {'flight_id':[], 'typecode':[]}

for aircraft in unique_aircrafts:
    mask = flights['typecode'] == aircraft
    optimal_id = flights[mask].sort_values('fuel')['flight_id'].iloc[0]
    optimal_flights['flight_id'] += [optimal_id]
    optimal_flights['typecode'] += [aircraft]
    
optimal_flights = pd.DataFrame(optimal_flights)

cursor.execute("DROP TABLE IF EXISTS optimal_flights;")

# Create the new table
create_table_query = f'''
    CREATE TABLE optimal_flights (
        flight_id TEXT PRIMARY KEY,
        typecode TEXT
    );
'''

cursor.execute(create_table_query)


for index, row in optimal_flights.iterrows():
    # Preparing the data to be inserted
    new_columns = ['flight_id', 'typecode']
    insert_data = tuple(row[col] for col in new_columns)

    # Creating query to insert new values
    insert_query = f'''
        INSERT INTO optimal_flights ({', '.join(new_columns)})
        VALUES ({', '.join('?' * len(insert_data))})
        ON CONFLICT(flight_id) DO UPDATE SET
        {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
    '''
    print(insert_data)
    cursor.execute(insert_query, insert_data)
        
    conn.commit()
conn.close()

print("Done")

('a3b414_1673405360_1673413567_KDEN_KSEA', 'A20N')
('ab1644_1672603218_1672611926_KDEN_KSEA', 'A319')
('a53774_1673913042_1673921355_KDEN_KSEA', 'A320')
('a12429_1672944919_1672952996_KDEN_KSEA', 'B38M')
('a448d3_1673405297_1673413729_KDEN_KSEA', 'B39M')
('ad01ab_1672753383_1672761742_KDEN_KSEA', 'B737')
('abd8a5_1674229884_1674238155_KDEN_KSEA', 'B738')
('a44565_1673808290_1673816496_KDEN_KSEA', 'B739')
('a01037_1672602317_1672611355_KDEN_KSEA', 'BCS1')
('a34e9d_1672755010_1672763243_KDEN_KSEA', 'BCS3')
('a32fd5_1672802591_1672811573_KDEN_KSEA', 'E75L')
('a7a90e_1672860194_1672868980_KDEN_KSEA', 'MD11')
('a35254_1672842175_1672851389_KDEN_KSEA', 'Other')
Done
